In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm
from sklearn.linear_model import ElasticNet
import joblib

import random
import datetime


In [3]:
df = pd.read_csv('../datasets/energy_consumption.csv')


In [18]:
df['machine_id'].unique()

array(['M3', 'M1', 'M2', 'M4', 'M5'], dtype=object)

In [21]:
dict_model = {}

for machine in df['machine_id'].unique():
    
    df_tmp = df.query(f'machine_id == "{machine}"').copy()
    X = df_tmp.drop(['energy','condition','machine_id'],axis=1).copy()
    y = df_tmp['energy'].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

    model = ElasticNet()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    score = sm.r2_score(y_test,y_pred)
    dict_model[machine] = {'model':model, 'r2': score}


In [28]:
with open('../list_models.pkl','wb') as f_out:
    joblib.dump(dict_model,f_out)

In [156]:
df.to_csv('./datasets/energy_consumption.csv',index=False)

In [41]:
starting_date = pd.Timestamp(datetime.datetime.now()).round('H')

In [42]:
dates = pd.date_range(start=starting_date, end=starting_date+pd.Timedelta('7days'),freq='1H')

In [50]:
dict_plan = {}
for m in df['machine_id'].unique():
    select_sample = [random.choice([True, False]) for x in dates]
    dict_plan[m] = df.query(f'machine_id=="{m}"').sample(len(dates)).iloc[select_sample,:]

In [51]:
dict_plan[m]

,energy,condition,volume,weight,machine_id,speed
456,1072.1,C000E,114.0,18.0,M5,9.3
2414,898.5,C000Y,96.0,15.0,M5,11.1
445,1184.8,C000W,125.0,19.0,M5,8.4
2482,1359.8,C000W,141.0,20.0,M5,7.4
3314,953.2,C000AY,103.0,17.0,M5,10.5
...,...,...,...,...,...,...
420,1127.0,C000W,120.0,19.0,M5,8.9
2447,1179.2,C000V,125.0,19.0,M5,8.5
3263,1181.1,C000J,125.0,20.0,M5,8.4
3433,1116.0,C000X,116.0,20.0,M5,8.9
